In [15]:
import pandas as pd
import numpy as np

In [24]:
data_path = "../data/static/yourfeed vs qualtrics_November 2, 2021_08.52.csv"
exp_id = '6676'

raw = pd.read_csv(data_path)
raw = raw.iloc[2: , :]
raw = raw[((raw['experiment_id']==exp_id) )]
raw['finish'] = (~raw['debug_url'].isna()) |  raw['complete']==1

/opt/homebrew/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (2,4,5,6,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,70,71,72,73,74,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,239,240,241,242) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(

In [25]:
raw['yourfeed'].value_counts()

1    1009
0     987
Name: yourfeed, dtype: int64

In [26]:
qualtrics = raw[(raw['yourfeed']=='0') & ((raw['experiment_id']==exp_id) )]

In [27]:
seedling = []
for i,r in qualtrics.iterrows():
    los=r['loopOrder'].split(",")[:-1]
    if len(los)>1:
        for i,lo in enumerate(los):
            col = "{}_response".format(int(lo.replace("e",""))+1)
            dat = (r['rid'], lo.replace("e",""), r[col], i+1, ("Android" in r['Browser_Operating System']) | ("iPhone" in r['Browser_Operating System']), r['experiment_id'], r['complete'], 0)
            seedling.append(dat)
bpd = pd.DataFrame(seedling, columns =["rid","item","response","order", "is_mobile", "experiment_id", 'completed', 'duration'])

In [28]:
(bpd['response'].fillna(0)).value_counts()

0    19185
1     9896
Name: response, dtype: int64

In [29]:
bpd['response'] = (bpd['response'].fillna(0)).astype(int)
bpd['dwell'] = -1 #.groupby("rid").mean()
bpd['condition'] =0
bpd['completed']= bpd['completed'].fillna(0)
bpd['completed'].value_counts()

1    28652
0      429
Name: completed, dtype: int64

In [30]:
import sys
sys.path.append("..")
from utils import dump_data, render_data, fx
# from creds import qualtrics_token
import pandas as pd
import psycopg2
from datetime import datetime
# import matplotlib
import numpy as np
# import matplotlib.pyplot as plt
import re
# %matplotlib inline  
import time
from creds import rds_url, rds_port, rds_db, rds_username, rds_password, user, password
import pandas as pd
import pandas.io.sql as sqlio

In [31]:
conn = psycopg2.connect(
    host=rds_url,
    port=rds_port,
    dbname=rds_db,
    user=rds_username,
    password=rds_password)

In [32]:
demo_dat = sqlio.read_sql_query("select * from demographics where experiment_id='6676'", conn)

In [33]:
user_id2rid = demo_dat[demo_dat['field']=='rid'][['value','user_id', 'user_agent', 'experiment_id', 'time_stamp']]
#user_id2rid[user_id2rid.duplicated(keep=False)] #must check
user_id2rid.columns = ['rid','user_id', 'user_agent', 'experiment_id', 'start_time']
user_id2rid = user_id2rid[~user_id2rid.duplicated(['user_id'])]
order_data = demo_dat[demo_dat['field']=='order'][['value','user_id']]
order_data = order_data[~order_data.duplicated(['user_id'])]
response=[]
for i,r in order_data.iterrows():
    for i,item in enumerate(r['value'].split(",")):
        response.append((i+1, item,r['user_id']))
response = pd.DataFrame(response, columns = ['order','item','user_id'])
yourfeed = pd.merge(response, user_id2rid, on='user_id', how='left')

s = (yourfeed.groupby('rid')['user_id'].nunique() > 1) #why??
mrid = s[s].index.values
yourfeed = yourfeed[~yourfeed['rid'].isin(mrid)]

In [34]:
len(set(yourfeed['rid']))

674

In [35]:
stim_dat = sqlio.read_sql_query("select * from stimulus where (experiment_id='6676') and (stim != 'introCard[dwell]' and stim != 'timebook[introCard][dwell]')", conn)
stim_dat['item'] =stim_dat['stim'].apply(lambda x:x.replace("][dwell]","").replace("card","").replace("timebook[",""))
stim_dat['dwell'] = stim_dat['stim'].str.contains("dwell")
dwells =  stim_dat[stim_dat['dwell']]
dwells['dwell'] = dwells['response'].astype(int)
dwells = dwells[~dwells.duplicated(subset=['user_id',"item"])]
dwells['item'] = dwells['item'].map(lambda x:x.replace("timebook[",'').replace("]",""))
#dwells[dwells.duplicated(subset=['user_id',"item"])].groupby(['user_id',"item"]).agg({"dwell" : np.var}) #check!

yourfeed= pd.merge(yourfeed,dwells[['user_id','item' ,'dwell']], on=['user_id','item'], how='left')

<ipython-input-35-3b84f4152eb2>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dwells['dwell'] = dwells['response'].astype(int)


In [36]:
s = stim_dat[~stim_dat['dwell']]
s = s[~s.duplicated(subset=['user_id',"item"])]
yourfeed = pd.merge(yourfeed,s[['user_id','item','response']], on =['user_id',"item"], how='left')
yourfeed['response'] = yourfeed['response'].fillna("-1").astype(int)+1
yourfeed['condition']= 1


In [37]:
did_dwell = demo_dat[demo_dat['field']=='did_dwell'][['user_id', 'time_stamp']]
did_dwell.columns = ['user_id', 'end_time']
did_dwell = did_dwell[~did_dwell.duplicated(subset=['user_id'])]
did_dwell['completed'] = 1
yourfeed = pd.merge(yourfeed,did_dwell, on =['user_id'], how='left')


In [38]:
yourfeed['is_mobile'] = yourfeed['user_agent'].str.contains("Android") | yourfeed['user_agent'].str.contains("iPhone") 
yourfeed['duration'] = (yourfeed['end_time'] - yourfeed['start_time']).map(lambda x:x.total_seconds())

In [39]:
long = pd.concat([yourfeed[['rid', 'item', 'response', 'order', 'dwell', 'condition', 'is_mobile', 'experiment_id','completed', 'duration']],bpd])
# long = yourfeed[['rid', 'item', 'response', 'order', 'dwell', 'condition', 'is_mobile', 'experiment_id','completed', 'duration']]

In [40]:
demos = raw[['rid', 'crt_ages', 'crt_printer', 'crt_bread', 'crt_race','crt_sheep','pk1','pk2','pk3','pk4','pk5','age','gender','education','income','ethnicity','demrep_c', 'social_media_check', 'facebook_frequency', 'media1', 'media2',       'newsfeed_knowledge', 'accuracy_imp', 'att1','att2', 'Duration (in seconds)','StartDate', 'debug_url']]
out = pd.merge(long,demos,on='rid',how='left')
out['completed'] = out['completed'].fillna(0)
out['time'] = out['duration'].astype(float) + out['Duration (in seconds)'].astype(float)
out['dwell'] = out['dwell'].fillna(-1).astype(int)

In [41]:
out.to_csv("../data/generated/soft_long.csv")

<h1>Comparative Dropout</h1>

In [69]:
#Start Qualtrics
raw['yourfeed'].value_counts()

0.0    1142
1.0    1134
Name: yourfeed, dtype: int64

In [70]:
#Finish Qualtrics
print(raw.groupby("yourfeed").agg({"finish":sum}))

          finish
yourfeed        
0.0        716.0
1.0        745.0


In [74]:
#Land on Yourfeed
demo_dat = sqlio.read_sql_query("select * from demographics where experiment_id='6676'", conn)
print("land on yourfeed: {}".format(len(set(demo_dat[demo_dat['field']=='order']['user_id']))))

land on yourfeed: 710


In [75]:
#Yourfeed Checkpoint
checkpoint = demo_dat[demo_dat['field']=='checkpoint'].groupby("user_id").agg({"value":max})
print(len(checkpoint))
checkpoint['value'].value_counts()

671


9    494
0     71
8     28
1     23
3     12
2     11
4     11
7     10
6      6
5      5
Name: value, dtype: int64

In [76]:
raw['nat'] = raw['att1'].isna()
last_question = []
for i,r in raw.iterrows():
    if r['finish']:
        last_question.append("1 finished")
    elif not np.isnan(r['accuracy_imp']):
        last_question.append("2 acc_imp")
    elif not np.isnan(r['crt_ages']):
        last_question.append("4 crt")
    elif not r['nat']:
        last_question.append("5 attc1")
    elif not('1' in str(r['Q1573']) or '2' in str(r['Q1573']) or '4' in str(r['Q1573'])):
        last_question.append("7 social media")
    else:
        last_question.append("7 social media")
    
raw['last_question'] = last_question

In [77]:
raw.groupby(['yourfeed','last_question']).size()

yourfeed  last_question 
0.0       1 finished        716
          2 acc_imp          39
          4 crt              25
          5 attc1           156
          7 social media    206
1.0       1 finished        745
          2 acc_imp           2
          4 crt              31
          5 attc1           145
          7 social media    211
dtype: int64

In [78]:
print("yourfeed : {}".format(1134 - 211-145 -31-2 ))
print("qualtrics : {}".format(1142-206-156-25-39))

yourfeed : 745
qualtrics : 716


In [80]:
raw.to_csv("../data/yvq_user_level.csv")

In [84]:
raw.reset_index().sample(frac=1).index

Int64Index([ 616,  229, 1159,   89,  117, 1286,  219, 1308,   66, 2270,
            ...
             957, 1514, 1163, 1614,  493, 1395, 2122, 1939,  819, 2003],
           dtype='int64', length=2276)